In [9]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [18]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.fc1 = nn.Linear(20 * 5 * 5, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 20 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
if torch.cuda.device_count() > 1:
    print('\n===> Training on GPUS!')
    net = nn.DataParallel(net)

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Net(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=500, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)
cuda:0


In [21]:
for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.061
[1,  4000] loss: 1.055
[1,  6000] loss: 1.004
[1,  8000] loss: 1.037
[1, 10000] loss: 1.004
[1, 12000] loss: 1.016
[2,  2000] loss: 0.878
[2,  4000] loss: 0.887
[2,  6000] loss: 0.882
[2,  8000] loss: 0.884
[2, 10000] loss: 0.884
[2, 12000] loss: 0.879
[3,  2000] loss: 0.743
[3,  4000] loss: 0.726
[3,  6000] loss: 0.762
[3,  8000] loss: 0.774
[3, 10000] loss: 0.789
[3, 12000] loss: 0.791
[4,  2000] loss: 0.585
[4,  4000] loss: 0.648
[4,  6000] loss: 0.662
[4,  8000] loss: 0.682
[4, 10000] loss: 0.651
[4, 12000] loss: 0.694
[5,  2000] loss: 0.483
[5,  4000] loss: 0.520
[5,  6000] loss: 0.550
[5,  8000] loss: 0.563
[5, 10000] loss: 0.588
[5, 12000] loss: 0.619
[6,  2000] loss: 0.390
[6,  4000] loss: 0.421
[6,  6000] loss: 0.455
[6,  8000] loss: 0.480
[6, 10000] loss: 0.512
[6, 12000] loss: 0.503
[7,  2000] loss: 0.323
[7,  4000] loss: 0.352
[7,  6000] loss: 0.395
[7,  8000] loss: 0.427
[7, 10000] loss: 0.439
[7, 12000] loss: 0.464
[8,  2000] loss: 0.261
[8,  4000] 

In [22]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the 10000 test images: 65 %
Accuracy of plane : 74 %
Accuracy of   car : 73 %
Accuracy of  bird : 53 %
Accuracy of   cat : 49 %
Accuracy of  deer : 67 %
Accuracy of   dog : 39 %
Accuracy of  frog : 77 %
Accuracy of horse : 68 %
Accuracy of  ship : 77 %
Accuracy of truck : 78 %
